# SQL

Check out: http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html  
And: https://www.dataquest.io/blog/python-pandas-databases/

In [1]:
# Importing relevant libraries
import pandas as pd
import sqlite3

In [5]:
# Create a connection and a cursor for querying
conn = sqlite3.connect("../data/fishbase_data.sqlite")
c = conn.cursor()

In [7]:
# Print out table names from the database
c.execute("SELECT name FROM sqlite_master WHERE type='table'")
c.fetchall()

[('fishbase_temperature',),
 ('fishbase_environment',),
 ('fishbase_extra_data',),
 ('seq_COI',),
 ('seq_CYTB',),
 ('seq_ND5',)]

In [8]:
# Reading in a table or SQL query directly into a pandas dataframe
fishbase_environment = pd.read_sql_query("SELECT * from fishbase_environment", conn)

In [9]:
type(fishbase_environment)

pandas.core.frame.DataFrame

In [10]:
fishbase_environment.head()

,index,species,climate,environment,depth_min,depth_max
0,0,Micropterus floridanus,subtropical,freshwater benthopelagic,NaN,NaN
1,1,Ischikauia steenackeri,temperate,freshwater demersal,NaN,NaN
2,2,Pseudohemiculter dispar,subtropical,freshwater benthopelagic brackish,NaN,NaN
3,3,Tinca tinca,temperate,freshwater brackish potamodromous demersal,1.0,NaN
4,4,Polypterus weeksii,tropical,freshwater demersal,NaN,NaN


In [57]:
# We can also get the column names as a pandas dataframe
pd.read_sql_query("SELECT * FROM fishbase_environment LIMIT 0", conn)

,index,species,climate,environment,depth_min,depth_max


In [11]:
# Reading in the fishbase_temperature- table
fishbase_temperature = pd.read_sql_query("SELECT * from fishbase_temperature", conn)

In [12]:
fishbase_temperature.head()

,index,species,temperature_min,temperature_max,climate,environment
0,0,Auxis thazard,27.0,28.0,tropical,oceanodromous marine pelagic-neritic
1,1,Percocypris pingi,NaN,NaN,subtropical,freshwater benthopelagic
2,2,Hiodon tergisus,NaN,NaN,temperate,pelagic freshwater
3,3,Halichoeres tenuispinis,NaN,NaN,tropical,reef-associated marine
4,4,Cottus hangiongensis,NaN,NaN,temperate,catadromous freshwater marine brackish demersal


In [13]:
# It is good practice to close the connection to the database once we have finished using it
conn.close()

# Joining tables

## Concatenating

In [14]:
# Along rows
fishbase_env_long = pd.concat([fishbase_environment, fishbase_environment], axis = 0)
fishbase_env_long.shape

(2530, 6)

In [70]:
# Along columns
fishbase_env_wide = pd.concat([fishbase_environment, fishbase_environment], axis = 1)
fishbase_env_wide.shape

(1265, 12)

## Inner join

In [15]:
fishbase_temp_env = pd.merge(left = fishbase_environment, 
    right = fishbase_temperature, left_on = 'species', right_on = 'species', how = 'inner')

# This can be shortened
fishbase_temp_env = pd.merge(fishbase_environment, fishbase_temperature, on = 'species', how = 'inner')

In [16]:
fishbase_temp_env.head()

,index_x,species,climate_x,environment_x,depth_min,depth_max,index_y,temperature_min,temperature_max,climate_y,environment_y
0,0,Micropterus floridanus,subtropical,freshwater benthopelagic,NaN,NaN,299,NaN,NaN,subtropical,freshwater benthopelagic
1,1,Ischikauia steenackeri,temperate,freshwater demersal,NaN,NaN,1027,NaN,NaN,temperate,freshwater demersal
2,2,Pseudohemiculter dispar,subtropical,freshwater benthopelagic brackish,NaN,NaN,981,NaN,NaN,subtropical,freshwater benthopelagic brackish
3,3,Tinca tinca,temperate,freshwater brackish potamodromous demersal,1.0,NaN,52,4.0,24.0,temperate,freshwater brackish potamodromous demersal
4,4,Polypterus weeksii,tropical,freshwater demersal,NaN,NaN,705,NaN,NaN,tropical,freshwater demersal


In [93]:
fishbase_temp_env.shape

(1265, 11)

In [95]:
# We can drop and rename duplicate columns
print(fishbase_temp_env.columns)

Index(['species', 'climate_x', 'environment_x', 'depth_min', 'depth_max',
       'temperature_min', 'temperature_max'],
      dtype='object')


In [99]:
fishbase_temp_env.drop(['index_x', 'index_y', 'climate_y', 'environment_y'], axis=1, inplace=True)
fishbase_temp_env.rename(columns = { 'climate_x' : 'climate', 'environment_x' : 'environment'}, inplace=True)

In [101]:
# Alternatively, we could have specified the columns to include during the join


In [109]:
fishbase_temp_env = pd.merge(fishbase_environment[['species', 'climate', 'environment', 'depth_min', 'depth_max']], 
    fishbase_temperature[['species', 'temperature_min', 'temperature_max']], on = 'species', how = 'inner')

In [114]:
fishbase_temp_env.head()

,species,climate,environment,depth_min,depth_max,temperature_min,temperature_max
0,Micropterus floridanus,subtropical,freshwater benthopelagic,NaN,NaN,NaN,NaN
1,Ischikauia steenackeri,temperate,freshwater demersal,NaN,NaN,NaN,NaN
2,Pseudohemiculter dispar,subtropical,freshwater benthopelagic brackish,NaN,NaN,NaN,NaN
3,Tinca tinca,temperate,freshwater brackish potamodromous demersal,1.0,NaN,4.0,24.0
4,Polypterus weeksii,tropical,freshwater demersal,NaN,NaN,NaN,NaN


# Plotting with Matplotlib